In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

# from __future__ import absolute_import
import sys
sys.path.append("../")

import os
from copy import copy

import numpy as np
import pandas as pd

from matplotlib import pylab as plt
import seaborn as sns

from random import choice

from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import Dense, Activation,Flatten,Convolution1D,Convolution2D,LSTM
from keras.regularizers import l2
from keras.optimizers import Adadelta,Adagrad,Adam,Adamax,RMSprop

from scipy.stats import randint as sp_randint

from utils.evaluation_utils import rmsle,log_pandas,inv_log_pandas
from utils.generic_utils import pickle_out,pickle_in
from utils.deep_learning_utils import ReportRmsleError
import utils.preprocessing_utils as prep

Using Theano backend.


In [2]:
(X_train,Y_train) = pickle_in(os.path.join("../","datasets","generated_features","train_binned.pkl"))
(X_valid,Y_valid) = pickle_in(os.path.join("../","datasets","generated_features","valid_binned.pkl"))
(X_test,Y_test) = pickle_in(os.path.join("../","datasets","generated_features","test_binned.pkl"))

In [3]:
mms = MinMaxScaler(feature_range=(0, 1))
X_train = mms.fit_transform(X_train)
X_valid = mms.transform(X_valid)
X_test = mms.transform(X_test)

stp_gen = prep.DataToTimeStamps(stamp_size=10,dim_out=4)
X_train = stp_gen.fit_transform(X_train)
X_valid = stp_gen.fit_transform(X_valid)
X_test = stp_gen.fit_transform(X_test)

Y_train = Y_train.apply(log_pandas).values.ravel()
Y_valid = Y_valid.apply(log_pandas).values.ravel()
# Y_train = Y_train.values.ravel()
# Y_valid = Y_valid.values.ravel()

print X_train.shape,X_valid.shape

(7452, 10, 1, 27) (1722, 10, 1, 27)


In [4]:
l2_reg = 0.0000000000001
l2_reg_dense = 0.0000000000001
# channels,features = 10,27
channels,rows,cols = 10,1,27

bike_model = Sequential()

#convo1
bike_model.add(Convolution2D(32, 1,3,init ="he_normal",
                             W_regularizer=l2(l=l2_reg),
#                              input_shape = (channels,features),
                             input_shape = (channels,rows,cols),
#                              border_mode = "same"
                            )
              )
bike_model.add(Activation('relu'))
# bike_model.add(MaxPooling2D(pool_size=(2, 2),strides = (1,1)))
# bike_model.add(Dropout(0.5))

#convo2
bike_model.add(Convolution2D(32, 1,3,init ="he_normal",
                             W_regularizer=l2(l=l2_reg),
#                              border_mode = "same"
                            )
              )
bike_model.add(Activation('relu'))
# bike_model.add(MaxPooling2D(pool_size=(2, 2),strides = (1,1)))
# bike_model.add(Dropout(0.5))
              
#flatten
bike_model.add(Flatten())

#dense1
bike_model.add(Dense(256,W_regularizer=l2(l=l2_reg_dense)))
bike_model.add(Activation('relu'))
# bike_model.add(Dropout(0.5))

#output
bike_model.add(Dense(1))

bike_model.compile(loss ="mse",
                  optimizer = Adagrad())

In [7]:
rms = ReportRmsleError(X_valid,Y_valid)
bike_model.fit(X_train, Y_train,
               validation_data=[X_valid,Y_valid], 
               batch_size=128, nb_epoch=1000, verbose=1, 
               callbacks=[rms],
               shuffle=False, class_weight=None, sample_weight=None)

RMSLE error: 1.72228461652
RMSLE error: 1.77264747376
RMSLE error: 1.80368796484
RMSLE error: 1.82807291852
RMSLE error: 1.83658319611
RMSLE error: 1.85117438538
RMSLE error: 1.86073891096
RMSLE error: 1.86935325388


KeyboardInterrupt: 

In [4]:
(X_train,Y_train) = pickle_in(os.path.join("../","datasets","generated_features","train_binned.pkl"))
(X_valid,Y_valid) = pickle_in(os.path.join("../","datasets","generated_features","valid_binned.pkl"))
(X_test,Y_test) = pickle_in(os.path.join("../","datasets","generated_features","test_binned.pkl"))

In [5]:
mms = MinMaxScaler(feature_range=(0, 1))
X_train = mms.fit_transform(X_train)
X_valid = mms.transform(X_valid)
X_test = mms.transform(X_test)

stp_gen = prep.DataToTimeStamps(stamp_size=10,dim_out=3)
X_train = stp_gen.fit_transform(X_train)
X_valid = stp_gen.fit_transform(X_valid)
X_test = stp_gen.fit_transform(X_test)

Y_train = Y_train.apply(log_pandas).values.ravel()
Y_valid = Y_valid.apply(log_pandas).values.ravel()
# Y_train = Y_train.values.ravel()
# Y_valid = Y_valid.values.ravel()

print X_train.shape,X_valid.shape

(7452, 10, 27) (1722, 10, 27)


In [6]:
l2_reg = 0.0000000000001
l2_reg_dense = 0.0000000000001

bike_lstm_model = Sequential()
bike_lstm_model.add(LSTM(10, input_dim=27))
bike_lstm_model.add(Dense(256))
bike_lstm_model.add(Dense(1))

bike_lstm_model.compile(loss ="mse",
                  optimizer = Adagrad())

In [8]:
rms = ReportRmsleError(X_valid,Y_valid)
bike_lstm_model.fit(X_train, Y_train,
               validation_data=[X_valid,Y_valid], 
               batch_size=128, nb_epoch=10000, verbose=1, 
               callbacks=[rms],
               shuffle=False, class_weight=None, sample_weight=None)

Train on 7452 samples, validate on 1722 samples
Epoch 1/10000
7424/7452 [============================>.] - ETA: 0s - loss: 2.0649

RMSLE error: 1.80490051972


7452/7452 [==============================] - 5s - loss: 2.0622 - val_loss: 2.4989
Epoch 2/10000
7424/7452 [============================>.] - ETA: 0s - loss: 1.4619

RMSLE error: 1.76170784402


7452/7452 [==============================] - 2s - loss: 1.4606 - val_loss: 1.9262
Epoch 3/10000
7424/7452 [============================>.] - ETA: 0s - loss: 1.2458

RMSLE error: 1.75631323055


7452/7452 [==============================] - 3s - loss: 1.2447 - val_loss: 1.5268
Epoch 4/10000
7424/7452 [============================>.] - ETA: 0s - loss: 1.0424

RMSLE error: 1.7671939296


7452/7452 [==============================] - 3s - loss: 1.0415 - val_loss: 1.2049
Epoch 5/10000
7424/7452 [============================>.] - ETA: 0s - loss: 0.8669

RMSLE error: 1.79317765146


7452/7452 [==============================] - 3s - loss: 0.8661 - 

ERROR: Internal Python error in the inspect module.
Below is the traceback from this internal error.


Unfortunately, your original traceback can not be constructed.



TypeError: 'NoneType' object is not iterable